Only the main process_and_describe function is shown.

This code below (generate_description_main.py) takes 3D models' downloaded files from the Thingiverse scraper and uses GenAI to generate: 
1. a description using either provided image files or blender snapshots
2. A category for the model
3. tags for the model

This outputs a csv with the generated information for each model and can be paused at any time (since it costs money).
This can be skipped as there are around 540 models in the csv, but can be run if more want to be loaded. The output csv has id,description,name,category,tags

In [1]:
# def process_and_describe(client, repo_owner, repo_name, local_extract_path, subdir, csv_writer, existing_ids, save_images=True):
#     """
#     Processes and describes each model in the repository.
# 
#     Parameters:
#     - client (OpenAIClient): Instance of OpenAIClient.
#     - repo_owner (str): Owner of the repository.
#     - repo_name (str): Name of the repository.
#     - local_extract_path (str): Path where files are extracted.
#     - subdir (dict): Information about the subdirectory.
#     - csv_writer (csv.writer): CSV writer object.
#     - existing_ids (set): Set of already processed object IDs.
#     - save_images (bool): Whether to save processed images.
#     """
#     subdir_path = os.path.join(local_extract_path, subdir['path'])
#     os.makedirs(subdir_path, exist_ok=True)
# 
#     try:
#         files = list_files_in_repo(repo_owner, repo_name, subdir['path'])
#     except Exception as e:
#         print(f"Error listing files in subdir {subdir['path']}: {e}")
#         return
# 
#     if not files:
#         print("No files found in the repository.")
#         return
# 
#     print(f"Found {len(files)} items in {subdir['path']} directory.")
#     for file_info in files:
#         if file_info['type'] == 'file' and file_info['name'].endswith('.zip'):
#             zip_path = os.path.join(subdir_path, file_info['name'])  # Path of zip file
#             try:
#                 download_file(file_info['download_url'], zip_path)  # Downloads file at path
#             except Exception as e:
#                 print(f"Skipping file due to download error: {zip_path} - {e}")
#                 continue
# 
#             extract_zip(zip_path, subdir_path)  # Extracts zip to subdir
#             images_dir = os.path.join(subdir_path, 'images')
#             image_files = find_image_files(images_dir)  # Finds all images for each item
# 
#             # Define the exclusion list
#             BROAD_TAGS_EXCLUSION = [
#                 "3d model", "fully assembled", "complete", "model", "assembly",
#                 "thing", "object", "product", "item", "prototype"
#             ]
# 
#             if not image_files:
#                 print(f"No image files found in {zip_path}. Attempting to render using Blender.")
#                 # Locate the .stl or .obj file under 'files/' subdirectory
#                 files_subdir = os.path.join(subdir_path, 'files')
#                 if not os.path.isdir(files_subdir):
#                     print(f"No 'files/' directory found in {zip_path}. Skipping.")
#                     continue
# 
#                 # Find all .stl and .obj files in 'files/' directory
#                 model_files = [f for f in os.listdir(files_subdir) if f.lower().endswith(('.stl', '.obj'))]
#                 if not model_files:
#                     print(f"No .stl or .obj files found in 'files/' directory of {zip_path}. Skipping.")
#                     continue
# 
#                 # For simplicity, process the first .stl/.obj file found
#                 model_file = os.path.join(files_subdir, model_files[0])
#                 print(f"Found model file: {model_file}")
# 
#                 # Define paths for Blender executable and script
#                 blender_executable = r"C:\Program Files\Blender Foundation\Blender 4.2\blender.exe"  # Update if different
#                 blender_script = 
#                 rendered_image_path = os.path.join(subdir_path, 'rendered_image.png')
# 
#                 # Call Blender to render the image
#                 rendered_image = render_model_with_blender(
#                     blender_executable,
#                     blender_script,
#                     model_file,
#                     rendered_image_path
#                 )
# 
#                 if rendered_image and os.path.exists(rendered_image):
#                     image_to_use = rendered_image
#                 else:
#                     print(f"Failed to render image for {zip_path}. Skipping.")
#                     continue
#             else:
#                 # Use the largest image file if no combined image is found
#                 image_to_use = find_largest_image_file(image_files)
#                 if not image_to_use:
#                     print(f"No suitable image found in {zip_path}")
#                     continue
# 
#             # Log the image being processed
#             print(f"Processing image: {image_to_use}")
# 
#             # Extract keywords from the zip file name and add new keywords
#             zip_file_name = os.path.splitext(file_info['name'])[0]
#             keywords = zip_file_name.replace('-', ' ').replace('_', ' ').split()
#             keywords.extend(["complete", "assembled", "whole", "IMG_"])  # Keywords include name and these
# 
#             # Generate initial guess based on the directory name
#             initial_guess = " ".join(keywords).capitalize()
# 
#             # Extract object_id from the zip file name
#             id_extracted = extract_id_from_name(zip_file_name)
#             if not id_extracted:
#                 print(f"Could not extract ID from zip file name: {zip_file_name}. Skipping.")
#                 continue
#             id = id_extracted
# 
#             # Check if object_id already exists
#             if id in existing_ids:
#                 print(f"Object ID {id} already exists in CSV. Skipping.")
#                 continue
# 
#             # Generate description, category, and tags using OpenAI
#             ai_data = client.describe_image_model(image_to_use, initial_guess)
#             if ai_data:
#                 description = ai_data.get('description', '').strip()
#                 category = ai_data.get('category', '').strip()
#                 tags = ai_data.get('tags', [])
# 
#                 if not description or not category or not tags:
#                     print(f"Incomplete AI data for {image_to_use}. Skipping.")
#                     continue
# 
#                 # Filter out broad/generic tags
#                 filtered_tags = filter_tags(tags, BROAD_TAGS_EXCLUSION)
# 
#                 # If no tags remain after filtering, assign a default tag
#                 if not filtered_tags:
#                     print(f"All tags for {image_to_use} were broad. Assigning default tag 'unknown'.")
#                     filtered_tags = ["unknown"]
# 
#                 # Format tags as comma-separated string
#                 tags_str = ", ".join(filtered_tags)
# 
#                 print(f"Description for {image_to_use}:\n{description}\n")
#                 print(f"Category: {category}")
#                 print(f"Tags: {tags_str}\n")
# 
#                 # Derive name from the zip file name or any other logic
#                 name = " ".join(keywords).capitalize()
# 
#                 # Write object_id, description, name, category, tags to CSV
#                 csv_writer.writerow([id, description, name, category, tags_str])
#                 print(f"Wrote data for Object ID {id} to CSV.\n")
# 
#                 # Add to existing_ids to prevent reprocessing within the same run
#                 existing_ids.add(id)
# 
#             # Save a copy of the image being processed for visual inspection if save_images is True
#             if save_images:
#                 try:
#                     output_dir = os.path.join(local_extract_path, 'processed_images')  # Puts them to processed_images
#                     os.makedirs(output_dir, exist_ok=True)
#                     img_output_path = os.path.join(output_dir, os.path.basename(image_to_use))  # Good for verifying correctness of GPT
# 
#                     # Ensure path length is within acceptable limits
#                     if len(img_output_path) > 255:
#                         print(f"Path too long: {img_output_path}")
#                         continue
# 
#                     # Save the image file
#                     with open(image_to_use, 'rb') as img_file:
#                         with open(img_output_path, 'wb') as output_file:
#                             output_file.write(img_file.read())
#                     print(f"Saved processed image to: {img_output_path}")
#                 except Exception as e:
#                     print(f"Error saving image {image_to_use}: {e}")
# 
#             # Pause to manage load
#             time.sleep(1)

Below is build_graph_with_AI.py, which constructs a knowledge graph using the AI descriptions, categories, and tag similarities. Because it extracts this information from the csv that is generated in the above code, make sure that the correct path is set.

### Only main function and output are shown.

This is the main system that was implemented for this entire project, as it uses AI and a knowledge graph to improve search functions.

In [1]:
# def main(csv_path, k_threshold=True, k=1):
#     """
#     Main function to orchestrate the semantic network construction.
# 
#     Parameters:
#     - csv_path (str): Path to the CSV file containing model descriptions.
#     - k_threshold (bool): If True, apply dynamic thresholding using mean + k*std.
#                            If False, do not apply dynamic thresholding.
#     - k (float): The multiplier for the standard deviation in threshold calculation.
#                Relevant only if k_threshold is True.
# 
#     The function performs the following steps:
#     1. Connects to Neo4j.
#     2. Loads and preprocesses data.
#     3. Extracts key components and expands synonyms.
#     4. Performs Named Entity Recognition.
#     5. Weights primary nouns in descriptions.
#     6. Calculates various similarity measures and combines them.
#     7. Constructs the knowledge graph in Neo4j with nodes and edges.
#     8. Applies additional optimizations like pruning and clustering.
#     """
#     print("Starting semantic network construction process.")
# 
#     # Step 1: Connect to Neo4j
#     graph_db = connect_to_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# 
#     # Step 2: Load Data
#     df = load_data(csv_path)
# 
#     # Step 3: Preprocess Data
#     df = preprocess_dataframe(df)
# 
#     # Step 4: Extract Key Components
#     df = extract_key_nouns_adjectives(df)
#     df, synonym_dict = expand_synonyms(df)
#     df = perform_ner(df)
# 
#     # Step 5: Preprocess Descriptions to Weight Primary Nouns and Incorporate Synonyms
#     df = duplicate_primary_nouns(df, weight=2.0)
# 
#     # Step 6: Text Similarity Calculation
#     tfidf_matrix, tfidf_vectorizer = compute_tfidf(df)
#     cosine_sim = compute_cosine_similarity_matrix(tfidf_matrix)
# 
#     # Compute Sentence-BERT similarity using weighted_description
#     embedding_sim = compute_sentence_transformer_similarity(df)
# 
#     # Compute Category similarity
#     category_sim = compute_category_similarity(df)
# 
#     # Compute Tag similarity
#     tag_sim = compute_tag_similarity(df)
# 
#     # Combine all similarities into a hybrid matrix
#     # Adjust weights as necessary to prioritize different components
#     final_sim = hybrid_similarity(
#         cosine_sim, embedding_sim, category_sim, tag_sim,
#         weight_cosine=0.3, weight_embedding=0.5,
#         weight_category=0.1, weight_tags=0.1
#     )
# 
#     # Adjust similarity scores based on primary nouns
#     final_sim = adjust_similarity_with_primary_nouns(df, final_sim, boost=1.2, penalize=0.8)
# 
#     # Adjust similarity scores based on entities
#     final_sim = adjust_similarity_with_entities(df, final_sim, boost=1.2, penalize=0.8)
# 
#     # Normalize the final similarity matrix
#     final_sim = normalize_similarity(final_sim)
# 
#     # Analyze similarity scores
#     analyze_similarity_scores(final_sim)
# 
#     # Determine dynamic threshold if enabled
#     if k_threshold:
#         dynamic_threshold = determine_threshold_mean_std(final_sim, k=k)  # Can adjust k as needed
#     else:
#         dynamic_threshold = 0.0  # No thresholding; connect all relevant edges
# 
#     # Visualize similarity distribution with threshold
#     visualize_similarity_distribution(
#         final_sim,
#         threshold=dynamic_threshold if k_threshold else None
#     )
# 
#     # Step 7: Knowledge Graph Construction with Embeddings
#     model = SentenceTransformer('all-MiniLM-L6-v2')  # Initialize Sentence-BERT model
#     create_nodes_with_embeddings(graph_db, df, model)
# 
#     # Step 8: Create Edges Based on Similarity
#     create_edges_batch(graph_db, df, final_sim, threshold=dynamic_threshold)  # Apply dynamic threshold
# 
#     # Step 9: Additional Optimizations
#     graph_pruning(
#         graph_db,
#         threshold=dynamic_threshold if k_threshold else 0.0
#     )  # Prune edges below the dynamic threshold
#     node_clustering(
#         graph_db, df, final_sim, threshold=dynamic_threshold
#     )  # Cluster based on dynamic threshold
# 
#     print("Knowledge graph construction complete.")
# 
# 
# if __name__ == "__main__":
#     # Set k_threshold=True to enable dynamic thresholding
#     # Set k_threshold=False to disable dynamic thresholding
#     # Adjust k as needed (e.g., k=1 for mean + 1*std)
#     main(CSV_PATH, k_threshold=True, k=2)

C:\Users\noski\PycharmProjects\Semantic_Search_Thingiverse\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


SpaCy Pipeline Components: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Starting semantic network construction process.
An unexpected error occurred while connecting to Neo4j: Cannot connect to any known routers
Loaded data with 539 records.
Starting text preprocessing...


Preprocessing Text: 100%|██████████| 539/539 [00:03<00:00, 177.12it/s]


Completed text preprocessing.
Extracting key nouns and adjectives with primary noun identification...
Completed extraction of key nouns and adjectives with multiple primary nouns.
Expanding nouns and adjectives with filtered synonyms...


Generating Synonyms: 100%|██████████| 2385/2385 [00:01<00:00, 1231.01it/s]


...

Error inserting batch of relationships: 'NoneType' object has no attribute 'begin'
Inserted batch of 1000 relationships.
Error inserting batch of relationships: 'NoneType' object has no attribute 'begin'
Inserted batch of 1000 relationships.
Error inserting batch of relationships: 'NoneType' object has no attribute 'begin'
Inserted batch of 1000 relationships.
Error inserting batch of relationships: 'NoneType' object has no attribute 'begin'
Inserted batch of 1000 relationships.
Error inserting batch of relationships: 'NoneType' object has no attribute 'begin'
Inserted final batch of 909 relationships.
Total edges created: 4909
Completed edge creation in Neo4j.
Pruning graph edges with similarity below 0.3663047495545203...
Error during graph pruning: 'NoneType' object has no attribute 'run'
Performing node clustering using NetworkX and Louvain method...


Adding Edges to Graph: 100%|██████████| 144991/144991 [00:00<00:00, 2947810.87it/s]


Detected 30 communities.
Updating Neo4j nodes with community labels...


Updating Communities:   0%|          | 0/1 [00:00<?, ?it/s]

Error during node clustering: 'NoneType' object has no attribute 'run'
Knowledge graph construction complete.


Lastly, below is the code that allows the user to search for their specific model (model_search.py) using the knowledge graph constructed above. There is also filtering, selection, research, and other useful functionalities.

### Only main function and output are shown.


In [2]:
# def main():
#     """
#     Main function to handle user queries and perform search on the knowledge graph with cascading filtering.
#     """
#     print("Initializing Semantic Search on Knowledge Graph...")
# 
#     # Connect to Neo4j
#     graph_db = connect_to_neo4j(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# 
#     # Fetch all models with embeddings
#     df_models = fetch_all_models(graph_db)
# 
#     # Extract embeddings as a 2D numpy array
#     model_embeddings = np.stack(df_models['embedding'].values)
# 
#     # Load Sentence Transformer model
#     print("Loading SentenceTransformer model...")
#     model = SentenceTransformer('all-MiniLM-L6-v2')
#     print("Model loaded successfully.")
# 
#     while True:
#         # Prompt user for input
#         user_query = input("\nEnter your search query (or type 'exit' to quit): ").strip()
#         if user_query.lower() == 'exit':
#             print("Exiting Semantic Search. Goodbye!")
#             break
#         if not user_query:
#             print("Empty query. Please enter a valid search term.")
#             continue
# 
#         # Preprocess the query with dynamic prioritization
#         preprocessed_query = preprocess_text(user_query)
#         if not preprocessed_query:
#             print("Failed to preprocess the query. Please try a different input.")
#             continue
# 
#         # Encode the query
#         query_embedding = model.encode(preprocessed_query, show_progress_bar=False)
# 
#         # Compute cosine similarity
#         print("Computing similarities...")
#         similarities = compute_cosine_similarity(query_embedding, model_embeddings)
# 
#         if similarities.size == 0:
#             print("Failed to compute similarities.")
#             continue
# 
#         # Display the top-k results
#         top_k = 10  # Number of top results to retrieve
#         top_results = display_results(df_models, similarities, top_k=top_k)
# 
#         if top_results is None or top_results.empty:
#             print("No similar models found.")
#             continue
# 
#         # Initialize the current search scope with the initial top results
#         current_scope = top_results
# 
#         while True:
#             # Prompt user to apply further filters or select a model
#             selected = prompt_filter(current_scope)
#             if selected == 'yes':
#                 # User wants to filter further
#                 # Prompt to select a model from current_scope
#                 try:
#                     selection = int(input(f"Select a model to filter by (1-{len(current_scope)}): ").strip())
#                     if 1 <= selection <= len(current_scope):
#                         selected_model = current_scope.iloc[selection - 1]
#                         selected_model_id = selected_model['id']
#                         print(f"\nSelected Model ID: {selected_model_id} - {selected_model['name'] if pd.notnull(selected_model['name']) else 'N/A'}")
#                     else:
#                         print(f"Please enter a number between 1 and {len(current_scope)}.")
#                         continue
#                 except ValueError:
#                     print("Invalid input. Please enter a valid number.")
#                     continue
# 
#                 # Fetch similar models to the selected model from the graph
#                 similar_df = fetch_similar_models_from_graph(graph_db, selected_model_id)
# 
#                 if similar_df.empty:
#                     print("No further models found based on your selection.")
#                     break  # Exit filtering loop
# 
#                 # Display the new set of similar models
#                 print(f"\nModels similar to Model ID: {selected_model_id}:")
#                 similar_top_k = 10  # Number of top similar models to display
#                 similar_top_results = display_results(similar_df, similar_df['similarity'].values, top_k=similar_top_k)
# 
#                 if similar_top_results is None or similar_top_results.empty:
#                     print("No similar models found in this step.")
#                     break  # Exit filtering loop
# 
#                 # Update the current scope to the new similar models
#                 current_scope = similar_top_results
# 
#             elif selected == 'no':
#                 # User does not want to filter further
#                 break  # Exit filtering loop
#             else:
#                 # User selected a model by rank to view its full summary
#                 selected_model_id = selected
#                 # Fetch the full details of the selected model
#                 try:
#                     query = """
#                     MATCH (m:Model {id: $selected_id})
#                     RETURN m.id AS id, m.name AS name, m.description AS description, m.category AS category, m.tags AS tags
#                     """
#                     record = graph_db.run(query, selected_id=selected_model_id).data()
#                     if not record:
#                         print(f"No details found for Model ID: {selected_model_id}.")
#                     else:
#                         model_details = record[0]
#                         print(f"\n--- Full Summary of Selected Model ---")
#                         print(f"ID: {model_details['id']}")
#                         print(f"Name: {model_details['name'] if pd.notnull(model_details['name']) else 'N/A'}")
#                         print(f"Category: {model_details['category'] if pd.notnull(model_details['category']) else 'N/A'}")
#                         print(f"Tags: {model_details['tags'] if pd.notnull(model_details['tags']) else 'N/A'}")
#                         print(f"Description: {model_details['description'] if pd.notnull(model_details['description']) else 'N/A'}")
#                         print(f"---------------------------------------\n")
#                 except Exception as e:
#                     print(f"Error fetching model details: {e}")
# 
#                 # After displaying the summary, ask if the user wants another search
#                 while True:
#                     another_search = input("Do you want to perform another search? (yes/no): ").strip().lower()
#                     if another_search in ['yes', 'y']:
#                         break  # Break to the main search loop
#                     elif another_search in ['no', 'n']:
#                         print("Exiting Semantic Search. Goodbye!")
#                         sys.exit(0)
#                     else:
#                         print("Please answer with 'yes' or 'no'.")
#                 break  # Exit filtering loop to perform another search
# 
# if __name__ == "__main__":
#     main()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\noski\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Initializing Semantic Search on Knowledge Graph...
Successfully connected to Neo4j.
Fetching all models and their embeddings from Neo4j...
Fetched 539 models with embeddings.
Loading SentenceTransformer model...


C:\Users\noski\PycharmProjects\Semantic_Search_Thingiverse\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model loaded successfully.
Computing similarities...

Top 10 similar models:

Rank 1:
ID: 4673220
Name: Cow farts 2 2551620 complete assembled whole img_
Category: Animals
Tags: holstein cow,livestock,farm animal,realistic texture,standing posture
Similarity Score: 0.5953
Description: This is a realistic rendering of a dairy cow, standing on all fours with a life-like posture. The cow features prominent black and white markings, typical of the Holstein breed. Its body is robust with a well-defined head, alert ears, and expressive eyes. The legs are sturdy, supporting its heavy frame, and the tail is accurately positioned with a natural curve. Textures on the cow simulate a smooth and realistic hide finish.

--------------------------------------------------
Rank 2:
ID: 4734034
Name: The cow 4811559 complete assembled whole img_
Category: Animals
Tags: cow,bovine,educational tool,decorative,animal figure,upright posture
Similarity Score: 0.5447
Description: This image features a three-d